In [1]:
import pandas as pd
import os


In [2]:
import spacy 
import subprocess
from string import punctuation

In [3]:
def get_hotwords(nlp, text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)

    return result # 5

In [5]:
from rake_nltk import Rake

In [6]:
r = Rake()
def rake_implement(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()

In [7]:
dir_path = os.path.dirname(os.path.realpath("__file__"))
df = pd.read_csv(dir_path + "/IMDb movies.csv")

In [8]:
df_nltk = df.copy()
df_nltk["key_words"] = df_nltk["description"].apply(lambda x: rake_implement(x,r))

TypeError: expected string or bytes-like object